In [ ]:

import pandas as pd

input_path = "OctopusGuard/data/experiment_address.csv"
df = pd.read_csv(input_path)

df["scam"] = 1
df.loc[df.index[-30:], "scam"] = 0

models = ["qwen", "llama"]
tasks = ["Kline_scam", "Tx_scam", "code_scam", "multimodal_scam"]

for model in models:
    for task in tasks:
        df[f"{model}_{task}"] = None  

output_path = "OctopusGuard/evaluations/backbone_comparison/backbone_comparison_template.csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')



In [ ]:
import pandas as pd
import json
import re
from openai import OpenAI
import time
from tqdm import tqdm

API_KEY = "sk-" 

file_paths = {
    "log_file": "OctopusGuard/evaluations/backbone_comparison/analysis_log_Llama32Vision.txt",
    "gt_json": "OctopusGuard/data/test_multimodal_data.json",
    "template_csv": "OctopusGuard/evaluations/backbone_comparison/backbone_comparison_template.csv"
}

client = OpenAI(
    api_key=API_KEY,
    base_url="https://api.deepseek.com"
)

api_cache = {}
def deepseek_judge_match(gt: str, model_output: str) -> str:
    cache_key = (gt, model_output)
    if cache_key in api_cache:
        return api_cache[cache_key]

    prompt = f"""
The following is the Ground Truth (human-labeled vulnerability description):
{gt}

And here is the vulnerability analysis output by the analysis tool:
{model_output}

As a blockchain security expert, please evaluate the following:
1. Does the tool's output cover all the core issues in the Ground Truth? Summarize your judgment in one sentence (Match, Partial Match, No Match).
2. Give a similarity score between 0 and 100 indicating how well the tool's output matches the Ground Truth. Please use the format: `Score: XX`
"""
    for attempt in range(3): 
        try:
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=[{"role": "user", "content": prompt.strip()}],
                max_tokens=256,
                stream=False
            )
            result = response.choices[0].message.content.strip()
            api_cache[cache_key] = result
            print(result)
            return result
        except Exception as e:
            print(f"  [API Error] Attempt {attempt + 1} failed: {e}. Retrying in 5 seconds...")
            time.sleep(5)
    
    print(f"  [API Error] All attempts failed for GT: '{gt[:50]}...'")
    return "Score: 0" 

def parse_model_log(log_path: str) -> dict:
    print(f"Parsing model log file: {log_path}")
    with open(log_path, 'r', encoding='utf-8') as f:
        content = f.read()

    results = {}
    blocks = re.split(r'==================== Contract Address: ', content)
    
    for block in tqdm(blocks, desc="Parsing Log Blocks"):
        if not block.strip():
            continue
        
        addr_match = re.match(r'(0x[a-fA-F0-9]{40})', block)
        if not addr_match:
            continue
        address = addr_match.group(1).lower()
        patterns = {
            'kline': r"🖼️ Analyzing token price chart image.*?A:(.*?)(?=📊 Analyzing transaction data)",
            'tx': r"📊 Analyzing transaction data.*?A:(.*?)(?=🔍 Analyzing smart contract code)",
            'code': r"📝 Contract Analysis Dialogue Log:.*?A:(.*?)(?=🧠 Final Assessment:)",
            'multimodal': r"🧠 Final Assessment:(.*?)(?======================= ANALYSIS COMPLETE)"
        }
        
        parsed_data = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, block, re.DOTALL | re.IGNORECASE)
            if match:
                parsed_data[key] = match.group(1).strip()
            else:
                parsed_data[key] = ""
        
        results[address] = parsed_data
    return results


def load_ground_truth(json_path: str) -> dict:
    print(f"Loading ground truth from: {json_path}")
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    gt_map = {}
    for i in range(0, len(data), 6):
        group = data[i:i+6]
        if len(group) == 6:
            gt_map[i // 6] = {
                'kline_gt': group[0]['completion'],
                'tx_gt': group[1]['completion'],
                'code_gt': group[4]['completion'],
                'multimodal_gt': group[5]['completion'], 
            }
    return gt_map

def is_scam_from_gt(gt_string: str, modality: str) -> bool:
    gt_lower = gt_string.lower()
    if modality == 'code':
        return "healthy" not in gt_lower
    else:
        return "scam: no" not in gt_lower

def is_scam_from_model_output(model_output_string: str, modality: str) -> bool:
    output_lower = model_output_string.lower()
    return "scam: no" not in output_lower

def main():
    print("Starting experiment 6 analysis for checkpoint-0...")
    
    df = pd.read_csv(file_paths["template_csv"])
    df['contract_address'] = df['contract_address'].str.lower()
    
    model_outputs = parse_model_log(file_paths["log_file"])
    ground_truths = load_ground_truth(file_paths["gt_json"])
    
    if len(df) != len(ground_truths) or len(df) > len(model_outputs):
         print(f"Warning: Data length mismatch!")
         print(f"CSV rows: {len(df)}, GT entries: {len(ground_truths)}, Logged addresses: {len(model_outputs)}")
    
    for i, row in tqdm(df.iterrows(), total=len(df), desc="Processing Contracts"):
        address = row['contract_address']
        
        if i not in ground_truths:
            print(f"Warning: No ground truth found for index {i}, address {address}. Skipping.")
            continue
        gt_data = ground_truths[i]

        if address not in model_outputs:
            print(f"Warning: No model output found for address {address}. Skipping.")
            continue
        model_output = model_outputs[address]

        gt_kline_is_scam = is_scam_from_gt(gt_data['kline_gt'], 'kline')
        model_kline_predicts_scam = is_scam_from_model_output(model_output.get('kline', ''), 'kline')
        df.loc[i, 'llama_Kline_scam'] = 1 if model_kline_predicts_scam else 0

        gt_tx_is_scam = is_scam_from_gt(gt_data['tx_gt'], 'tx')
        model_tx_predicts_scam = is_scam_from_model_output(model_output.get('tx', ''), 'tx')
        df.loc[i, 'llama_Tx_scam'] = 1 if model_tx_predicts_scam else 0
        
        gt_multi_is_scam = is_scam_from_gt(gt_data['multimodal_gt'], 'multimodal')
        model_multi_predicts_scam = is_scam_from_model_output(model_output.get('multimodal', ''), 'multimodal')
        df.loc[i, 'llama_multimodal_scam'] = 1 if model_multi_predicts_scam else 0

        gt_code = gt_data['code_gt']
        model_code = model_output.get('code', 'No output found.')
        
        gt_is_healthy = not is_scam_from_gt(gt_code, 'code')

        is_match_code = False
        if gt_is_healthy:
            if "no vulnerabilities" in model_code.lower() or "no indication" not in model_code.lower() and len(model_code) < 200:
                is_match_code = True
        else:
            api_response = deepseek_judge_match(gt_code, model_code)
            score_match = re.search(r'Score:\s*(\d+)', api_response, re.IGNORECASE)
            score = int(score_match.group(1)) if score_match else 0
            if score >= 50:
                is_match_code = True
        
        if i < 270:
            df.loc[i, 'llama_code_scam'] = 1 if is_match_code else 0
        else:
            df.loc[i, 'llama_code_scam'] = 0 if is_match_code else 1

    output_path = file_paths["template_csv"]
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"\nAnalysis complete! Results saved to: {output_path}")

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd

df = pd.read_csv("OctopusGuard/evaluations/backbone_comparison/backbone_comparison_template.csv")

gt_column = "scam"

results = []

for column in df.columns:
    if column in ["contract_address", gt_column]:
        continue

    y_true = df[gt_column]
    y_pred = df[column]

    tp = ((y_pred == 1) & (y_true == 1)).sum()
    fn = ((y_pred == 0) & (y_true == 1)).sum()
    fp = ((y_pred == 1) & (y_true == 0)).sum()
    tn = ((y_pred == 0) & (y_true == 0)).sum()

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0.0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    print(f"result - {column}")
    print(f"TP: {tp}")
    print(f"FN: {fn}")
    print(f"FP: {fp}")
    print(f"TN: {tn}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("-" * 40)

    results.append({
        "Model": column,
        "TP": tp,
        "FN": fn,
        "FP": fp,
        "TN": tn,
        "Precision": precision,
        "Recall": recall,
        "Accuracy": accuracy,
        "F1 Score": f1
    })

results_df = pd.DataFrame(results)
results_df.to_csv("backbone_comparison_results.csv", index=False)
